## Avaliando [Crash Course](https://mxnet.apache.org/versions/master/api/python/docs/tutorials/getting-started/crash-course/2-create-nn.html)


### Step2

In [25]:
import mxnet as mx
from mxnet import np, npx
from mxnet.gluon import nn

npx.set_np()

mx.__version__

'1.8.0'

## Step 2 - Create neural network
Uma camada com 3 entradas e 5 saidas, com função de ativação "RELU"


In [26]:
layer=nn.Dense(5,in_units=3,activation='relu')
layer

Dense(3 -> 5, Activation(relu))

In [27]:
layer.initialize()

#### dez linhas com três parametros

In [28]:
x=np.random.uniform(-1,1,(10,3))
x

array([[ 0.09709179,  0.27321362, -0.8518564 ],
       [-0.10710186,  0.8004246 ,  0.24689186],
       [-0.69020677,  0.9856076 , -0.9356934 ],
       [-0.14531201,  0.75238216, -0.42828953],
       [ 0.96896505, -0.8357303 , -0.31394452],
       [-0.00288934, -0.00425076, -0.7892191 ],
       [-0.45202208, -0.33587497, -0.6148413 ],
       [ 0.7164177 , -0.08857018, -0.73732615],
       [-0.5796278 ,  0.61682177, -0.12559462],
       [-0.229886  , -0.88089114, -0.61813056]])

In [29]:
layer(x)

array([[0.04491673, 0.00914199, 0.        , 0.02022739, 0.        ],
       [0.        , 0.02195932, 0.02917553, 0.03675443, 0.        ],
       [0.03693289, 0.04353196, 0.        , 0.01007843, 0.        ],
       [0.00141912, 0.02450657, 0.        , 0.03139923, 0.        ],
       [0.0364917 , 0.        , 0.        , 0.01445661, 0.04297189],
       [0.05382494, 0.00352589, 0.        , 0.        , 0.        ],
       [0.06406542, 0.00277784, 0.        , 0.        , 0.        ],
       [0.03985525, 0.        , 0.        , 0.03896289, 0.        ],
       [0.        , 0.02803061, 0.01426389, 0.        , 0.        ],
       [0.08207822, 0.        , 0.        , 0.        , 0.        ]])

In [30]:
layer.params

dense15_ (
  Parameter dense15_weight (shape=(5, 3), dtype=float32)
  Parameter dense15_bias (shape=(5,), dtype=float32)
)

#### pesos e bias de entrada de cada elemento da camada interna de 5 neuronios.

In [31]:
layer.weight.data()

array([[-0.01926848, -0.04049358, -0.0679116 ],
       [-0.01946345,  0.02622988, -0.00453758],
       [-0.0105041 ,  0.02191875,  0.04255389],
       [ 0.06167717,  0.0539865 ,  0.00059964],
       [ 0.0258387 , -0.03875457,  0.04603765]])

In [32]:
layer.bias.data()


array([0., 0., 0., 0., 0.])

Chain layers into a neural network using nn.Sequential

Sequential é a ligação de camadas onde a saída de uma camada é a entrada de outra

In [33]:
net = nn.Sequential()

net.add(nn.Dense(5,in_units=3,activation='relu'),nn.Dense(25,activation='relu'),nn.Dense(2))

net

Sequential(
  (0): Dense(3 -> 5, Activation(relu))
  (1): Dense(-1 -> 25, Activation(relu))
  (2): Dense(-1 -> 2, linear)
)

In [34]:
net[1]

Dense(-1 -> 25, Activation(relu))

#### Custom neural network architecture flexibly

In [35]:
class Net(nn.Block):
    def __init__(self):
        super().__init__()
        
    def forward(self,x):
        return x

In [36]:
class MLP(nn.Block):
    def __init__(self):
        super().__init__()
        self.dense1=nn.Dense(5,activation='relu')
        self.dense2=nn.Dense(25,activation='relu')
        self.dense3=nn.Dense(2)
        
    def forward(self,x):
        layer1=self.dense1(x)
        layer2=self.dense2(layer1)
        layer3=self.dense3(layer2)
        return layer3

In [37]:
net=MLP()
net

MLP(
  (dense1): Dense(-1 -> 5, Activation(relu))
  (dense2): Dense(-1 -> 25, Activation(relu))
  (dense3): Dense(-1 -> 2, linear)
)

In [38]:
net.dense1.params

dense19_ (
  Parameter dense19_weight (shape=(5, -1), dtype=float32)
  Parameter dense19_bias (shape=(5,), dtype=float32)
)

#### Creating custom layers using Parameters

In [39]:
from mxnet.gluon import Parameter

weight=Parameter("custom_parameter_weight",shape=(5,-1))
bias=Parameter("custom_parameter_bias",shape=(5,-1))

weight,bias

(Parameter custom_parameter_weight (shape=(5, -1), dtype=<class 'numpy.float32'>),
 Parameter custom_parameter_bias (shape=(5, -1), dtype=<class 'numpy.float32'>))

Custom layer com função linear sem função de ativação. 

w*x+b

In [40]:
class custom_layer(nn.Block):
    def __init__(self,out_units, in_units=0):
        super().__init__()
        
        # mxnet 2.0
        # self.weight=Parameter("weight",shape=(in_units,out_units),allow_deferred_init=True)
        # self.bias=Parameter("bias",shape=(out_units,),allow_deferred_init=True)
        
        # mxnet 1.8.0
        self.weight = self.params.get('weight', shape=(in_units, out_units),allow_deferred_init=True)
        self.bias = self.params.get('bias', shape=(out_units,),allow_deferred_init=True)

        
    def forward(self,x):
        return np.dot(x,self.weight.data())+self.bias.data()

In [41]:
dense=custom_layer(3,in_units=5)
dense.initialize()
dense(np.random.uniform(size=(4,5)))

RuntimeError: Parameter 'weight' has not been initialized. Note that you should initialize parameters and create Trainer with Block.collect_params() instead of Block.params because the later does not include Parameters of nested child Blocks

### [LeNet](http://yann.lecun.com/exdb/lenet/)

In [ ]:
class LeNet(nn.Block):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2D(channels=6,kernel_size=3,activation='relu')
        self.pool1=nn.MaxPool2D(pool_size=2,strides=2)
        self.conv2=nn.Conv2D(channels=16,kernel_size=3,activation='relu')
        self.pool2=nn.MaxPool2D(pool_size=2,strides=2)
        self.dense1=nn.Dense(120,activation='relu')
        self.dense2=nn.Dense(84,activation='relu')
        self.dense3=nn.Dense(10)
        
    def forward(self,x):
        x=self.conv1(x)
        x=self.pool1(x)
        x=self.conv2(x)
        x=self.pool2(x)
        x=self.dense1(x)
        x=self.dense2(x)
        x=self.dense3(x)
        return x

lenet=LeNet()        

In [ ]:
class LeNet_custom(nn.Block):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2D(channels=6,kernel_size=3,activation='relu')
        self.pool1=nn.MaxPool2D(pool_size=2,strides=2)
        self.conv2=nn.Conv2D(channels=16,kernel_size=3,activation='relu')
        self.pool2=nn.MaxPool2D(pool_size=2,strides=2)
        self.dense1=nn.Dense(120,activation='relu')
        self.dense2=nn.Dense(84,activation='relu')
        self.dense3=custom_layer(10,84)
        
    def forward(self,x):
        x=self.conv1(x)
        x=self.pool1(x)
        x=self.conv2(x)
        x=self.pool2(x)
        x=self.dense1(x)
        x=self.dense2(x)
        x=self.dense3(x)
        return x

lenet_custom=LeNet_custom()        

In [ ]:
image_data=np.random.uniform(-1,1,(1,1,28,28))

lenet.initialize()
lenet_custom.initialize()

print("LeNet:")
print(lenet(image_data))

print("Custom LeNet:")
print(lenet_custom(image_data))


In [ ]:
lenet.conv1.weight.data().shape, lenet.dense1.bias.data().shape,lenet_custom.conv1.weight.data().shape, lenet_custom.dense1.bias.data().shape


#### Using predefined (pretrained) architectures

[Gluon CV model zoo](https://cv.gluon.ai/model_zoo/index.html)

[Gluon NLP model zoo](https://nlp.gluon.ai/model_zoo/index.html)

In [ ]:
from mxnet.gluon import model_zoo

net=model_zoo.vision.resnet50_v2(pretrained=True)
net.hybridize()

dummy_input=np.ones(shape=(1,3,224,224))
output=net(dummy_input)
output.shape


### Deciding the paradigm for your network

In [ ]:
net_hybrid_seq=nn.HybridSequential()

net_hybrid_seq.add(nn.Dense(5,in_units=3,activation='relu'),
    nn.Dense(25,activation='relu'),nn.Dense(2))

net_hybrid_seq

In [ ]:
net_hybrid_seq.hybridize()

### Creating custom layers using Parameters(HbridBlocks API)

In [ ]:
class CustomLayer(nn.HybridBlock):
    def __init__(self,out_units, in_units=-1):
        super().__init__()
        self.weight=Parameter("weight",shape=(in_units, out_units),allow_deferred_init=True)
        self.bias=Parameter("bias",shape=(out_units,),allow_deferred_init=True)
        
    def forward(self, x):
        print(self.weight.shape,self.bias.shape)
        return np.dot(x,self.weight.data())+self.bias.data()

    def infer_shape(self,x):
        print(self.weight.shape,x.shape)
        self.weight.shape=(x. shape[-1],self.weight.shape[1])


dense=CustomLayer(3)

dense.initialize()
dense(np.random.uniform(size=(4,5)))
    

### Performance

In [ ]:
from time import time

def benchmark(net,x):
    y=net(x)
    start=time()
    for i in range(1,1000):
        y=net(x)
    return time()-start


x_bench=np.random.normal(size=(1,521))

net_hybrid_seq=nn.HybridSequential()

net_hybrid_seq.add (nn.Dense(256,activation='relu'),
                nn.Dense(128,activation='relu'),
                nn.Dense(2))

net_hybrid_seq.initialize()

print('Before hybridizing: %.4f sec'%(benchmark(net_hybrid_seq, x_bench)))
net_hybrid_seq.hybridize()
print('After hybridizing: %.4f'%(benchmark(net_hybrid_seq, x_bench)))

In [ ]:
from mxnet.gluon import HybridBlock

class MLP_Hybrid(HybridBlock):
    def __init__(self):
        super().__init__()
        self.dense1=nn.Dense(256,activation='relu')
        self.dense2=nn.Dense(123,activation='relu')
        self.dense3=nn.Dense(2)

    def forward(self,x):
        layer1=self.dense1(x)
        layer2=self.dense2(layer1)
        layer3=self.dense3(layer2)
        return layer3

net_hybrid=MLP_Hybrid()
net_hybrid.initialize()

print('Before hybridizing: %.4f sec'%(benchmark(net_hybrid, x_bench)))
net_hybrid.hybridize()
print('After hybridizing: %.4f'%(benchmark(net_hybrid, x_bench)))

#### Saving and Loading your models

In [ ]:
file_name='../../../data/crashCourse/models/layer.params'
layer.save_parameters(file_name)

In [ ]:
def build_model():
    layer=nn.Dense(5,in_units=3,activation='relu')
    return layer

layer_new=build_model()

In [ ]:
layer_new.load_parameters(file_name)

#### Save/load the model weights/parameters and the architectures

In [ ]:
net_hybrid.export('../../../data/crashCourse/models/MLP_hybrid')

In [ ]:
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    net_loaded=nn.SymbolBlock.imports('../../../data/crashCourse/models/MLP_hybrid-symbol.json',['data'], '../../../data/crashCourse/models/MLP_hybrid-0000.params',device=None)

In [ ]:
net_loaded(x_bench)

#### Visualizing Models

In [ ]:
layer.summary(x)

In [ ]:
lenet.summary(image_data)

In [ ]:
net_hybrid_summary=MLP_Hybrid()

net_hybrid_summary.initialize()

net_hybrid_summary.summary(x_bench)

net_hybrid_summary.hybridize()